In [1]:
# -*- coding: utf-8 -*-
import dataiku
from dataiku import spark as dkuspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StructType, StructField, StringType

import math
import pandas as pd
import numpy as np

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

/Users/rendo/Library/DataScienceStudio/dss_home/code-envs/python/Python37/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


ModuleNotFoundError: No module named 'pyspark'

In [3]:
# Read recipe inputs
fitness = dataiku.Dataset("FITNESS")
fitness_df = dkuspark.get_dataframe(sqlContext, fitness)

NameError: name 'dataiku' is not defined

In [0]:
fitness_df = fitness_df.dropna()

In [0]:
column_names = fitness_df.columns
corr_method = dataiku.get_custom_variables()["corr_method"]
vector_col = "corr_features"

In [0]:
corr_method

In [0]:
# convert to vector column first
assembler = VectorAssembler(inputCols=fitness_df.columns, outputCol=vector_col)
df_vector = assembler.transform(fitness_df).select(vector_col)

In [0]:
matrix = Correlation.corr(df_vector, vector_col, method=corr_method)

In [0]:
type(matrix)

In [0]:
matrix.show(truncate=False)

In [0]:
cor_np = matrix.collect()[0]["pearson({})".format(vector_col)].values
cor_np

In [0]:
type(cor_np)

In [0]:
dim = int(math.sqrt(len(cor_np)))
cor_mat = cor_np.reshape((dim,dim))

In [0]:
cor_mat

In [0]:
# Approach 1. - Failed
df_spark = sqlContext.createDataFrame(pd.DataFrame(cor_mat))
df_spark.show()

In [0]:
# Approach 2. - Failed
corr_data= pd.DataFrame(cor_mat, index= column_names,columns = column_names).reset_index()
corr_pyspark_df = spark.createDataFrame(corr_data)
corr_pyspark_df.show()

In [0]:
# Write recipe outputs
corr_pyspark = dataiku.Dataset("corr_pyspark")
dkuspark.write_with_schema(corr_pyspark, df_spark)